In [2]:
import os
import sys

from os.path import join, pardir
sys.path.append(pardir)

from bids import BIDSLayout
from itertools import product, chain

from nipype import config
from nipype.pipeline.engine import Workflow, Node
from ica_wf import make_subject_ica_wf

In [3]:
config.enable_debug_mode()

### Preproc & ICA Flow:

(For every dataset:)  
For every subject:  
For every run:  
For every task:  
For every space:  

      1. extract filenames of bold & mask file
      2. feed into preproc.py
      -> return ICs


In [58]:
def get_datapaths(bids_layout, subject, session, run, task, space, outdir='/LOCAL/jzerbe/temp_results/melodic'):
    """
    Extract mask and bold file paths for one subject for one task
    of one run for one type of space.
    
    Input: BIDSlayout, subject, ...  
    Output: bold_file, mask_file
    """
    # check if run and session are present
    run = None if run in ('0','00', '') else run
    session = None if session in ('0','00', '') else session
    # get paths
    bold_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='bold',
                        return_type='filename'
                        )
    bold_file = [i for i in bold_file if "AROMA" not in i]
    
    mask_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='mask',
                        return_type='filename'
                        )
    out_dir = join(outdir,
                   f'sub-{subject}',
                   f'sub-{subject}_ses-{session}_task-{task}_run-{run}_space-{space}-melodic')
    return bold_file, mask_file, out_dir

In [59]:
def read_paths(bids_layout, subject="all", session="all", run="all", task="all", space="all",
               outdir="/LOCAL/jzerbe/temp_results/melodic"):
    """
    Check if all data paths or only specific paths are asked for and give them back.
    
    Input: BIDSlayout
    Optional Input: subject, session, run, task, space
    Output: one file with all bold and mask file paths as tuples
    """
    if all([param == "all" for param in (subject, session, run, task, space)]):
        subject = bids_layout.get(return_type='id', target='subject', desc='preproc')
        session = bids_layout.get(return_type='id', target='session', desc='preproc')
        run = bids_layout.get(return_type='id', target='session', desc='preproc')
        task = bids_layout.get(return_type='id', target='task', desc='preproc')
        space = bids_layout.get(return_type='id', target='space', desc='preproc')
    else:
        subject = subject
        session = [session] # TODO: for many runs/sessions, check if pybids gives just a number or a list
        run = [run]
        task = [task]
        space = [space]
    
    # check if run and session are present
    session = '0' if session == [] else session
    run = '0' if run == [] else run
    
    # all combinations
    combinations = list(product(subject, session, run, task, space))
    boldfiles_nested, maskfiles_nested, outdirs = zip(*[
        get_datapaths(bids_layout, *params, outdir) for params in combinations
    ])
    boldfiles = [val for sublist in boldfiles_nested for val in sublist]
    maskfiles = [val for sublist in maskfiles_nested for val in sublist]
    
    return boldfiles, maskfiles, outdirs 

In [60]:
# create layout from BIDS dataset
bidsdata_dir = '/LOCAL/jzerbe/faces_vs_houses/ds002938'
layout = BIDSLayout(bidsdata_dir, derivatives=True)

In [61]:
# get all data paths
OUT_DIR = '/LOCAL/jzerbe/temp_results/melodic' # later change to /bidsdata_dir + /melodic
boldlist, masklist, outdirlist = read_paths(layout, outdir=OUT_DIR)

In [1]:
## testing: get data paths for one subject for certain parameters
#subjects = ['03'] 
#sessions = ['0'] # per subject
#runs = ['0']
#tasks = ['effort']
#spaces = ['T1w']

In [62]:
# parameter for melodic workflow
TR = 1.5
HPF = 120./TR
FWHM = 4.0
BASE_DIR = '/LOCAL/jzerbe/temp_results' # change to /bidsdata_dir?

In [63]:
# create a list with one workflow per bold/mask/output path
runwfs = []
i = 1
for boldfile, maskfile, outdir in zip(boldlist, masklist, outdirlist):
    print('\nITERATION: ', i) # check if it really creates them all
    runwf = make_subject_ica_wf()
    
    runwf.inputs.inputspec.hpf = HPF
    runwf.inputs.inputspec.tr = TR
    runwf.inputs.inputspec.fwhm = FWHM
    runwf.base_dir = BASE_DIR    
    runwf.inputs.inputspec.bold_file = boldfile
    runwf.inputs.inputspec.mask_file = maskfile
    runwf.inputs.inputspec.out_dir = outdir
    runwf.name = join(f'node_{i}')
    runwfs.append(runwf)
    i += 1



ITERATION:  1
220511-18:23:16,605 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.calcthresh): No edge data
220511-18:23:16,605 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.calcthresh): new edge data: {'connect': [('bold_file', 'boldfile'), ('mask_file', 'maskfile')]}
220511-18:23:16,606 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): No edge data
220511-18:23:16,606 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): new edge data: {'connect': [('bold_file', 'in_file'), ('fwhm', 'fwhm')]}
220511-18:23:16,607 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220511-18:23:16,607 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220511-18:23:16,607 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220511-18:23:16,607 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': 

220511-18:23:16,673 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.calcthresh): new edge data: {'connect': [('bold_file', 'boldfile'), ('mask_file', 'maskfile')]}
220511-18:23:16,674 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): No edge data
220511-18:23:16,674 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): new edge data: {'connect': [('bold_file', 'in_file'), ('fwhm', 'fwhm')]}
220511-18:23:16,674 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220511-18:23:16,675 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220511-18:23:16,675 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220511-18:23:16,676 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220511-18:23:16,676 nipype.workflow DEBUG:
	 (melod

220511-18:23:16,737 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): No edge data
220511-18:23:16,738 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): new edge data: {'connect': [('bold_file', 'in_file'), ('fwhm', 'fwhm')]}
220511-18:23:16,738 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220511-18:23:16,738 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220511-18:23:16,739 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220511-18:23:16,739 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220511-18:23:16,739 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220511-18:23:16,740 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_th

220511-18:23:16,835 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220511-18:23:16,835 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220511-18:23:16,836 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220511-18:23:16,836 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220511-18:23:16,837 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220511-18:23:16,837 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_thresh', 'brightness_threshold')]}
220511-18:23:16,837 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): No edge data
220511-18:23:16,838 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): new edge data: {'connect': [('smoothed_file', 'i

220511-18:23:16,917 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220511-18:23:16,917 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220511-18:23:16,917 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220511-18:23:16,918 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220511-18:23:16,918 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_thresh', 'brightness_threshold')]}
220511-18:23:16,918 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): No edge data
220511-18:23:16,919 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): new edge data: {'connect': [('smoothed_file', 'in_file')]}
220511-18:23:16,925 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): No e

...

220511-18:23:17,162 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): No edge data
220511-18:23:17,163 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): new edge data: {'connect': [('smoothed_file', 'in_file')]}
220511-18:23:17,163 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): No edge data
220511-18:23:17,163 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220511-18:23:17,165 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220511-18:23:17,167 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220511-18:23:17,170 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220511-18:23:17,172 nipype.workflow DEBUG:
	 [Node] inputspec - setting input bold_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-18/func/sub-18_task-effort_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz
220511-18:23:17,176 nipype.workflow DEB

220511-18:23:17,247 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): new edge data: {'connect': [('smoothed_file', 'in_file')]}
220511-18:23:17,247 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): No edge data
220511-18:23:17,248 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220511-18:23:17,251 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220511-18:23:17,262 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220511-18:23:17,265 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220511-18:23:17,268 nipype.workflow DEBUG:
	 [Node] inputspec - setting input bold_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-23/func/sub-23_task-1back_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz
220511-18:23:17,270 nipype.workflow DEBUG:
	 [Node] inputspec - setting input mask_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/deriv

220511-18:23:17,339 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): No edge data
220511-18:23:17,340 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220511-18:23:17,343 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220511-18:23:17,346 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220511-18:23:17,349 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220511-18:23:17,353 nipype.workflow DEBUG:
	 [Node] inputspec - setting input bold_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-23/func/sub-23_task-effort_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz
220511-18:23:17,355 nipype.workflow DEBUG:
	 [Node] inputspec - setting input mask_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-23/func/sub-23_task-effort_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
220511-18:23:17,358 nipype.workflow DEBUG:
	 [N

220511-18:23:17,495 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220511-18:23:17,499 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220511-18:23:17,503 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220511-18:23:17,507 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220511-18:23:17,510 nipype.workflow DEBUG:
	 [Node] inputspec - setting input bold_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-1back_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz
220511-18:23:17,512 nipype.workflow DEBUG:
	 [Node] inputspec - setting input mask_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-1back_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
220511-18:23:17,515 nipype.workflow DEBUG:
	 [Node] inputspec - setting input out_dir = /LOCAL/jzerbe/temp_results/melodic/sub-28/sub-28_ses-None

220511-18:23:17,593 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220511-18:23:17,597 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220511-18:23:17,600 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220511-18:23:17,604 nipype.workflow DEBUG:
	 [Node] inputspec - setting input bold_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-effort_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz
220511-18:23:17,606 nipype.workflow DEBUG:
	 [Node] inputspec - setting input mask_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-effort_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
220511-18:23:17,608 nipype.workflow DEBUG:
	 [Node] inputspec - setting input out_dir = /LOCAL/jzerbe/temp_results/melodic/sub-28/sub-28_ses-None_task-effort_run-None_space-MNI152NLin6Asym-melodic

ITERATION:  35
220511-18:23:17,610 nipype.workflow DEBUG:
	 (melodicwf.inputspec, m

In [64]:
dataset_wf = Workflow(name='dataset_wf')
dataset_wf.add_nodes(runwfs)

In [65]:
dataset_wf.run(plugin='MultiProc', plugin_args=dict(n_procs=30))

220511-18:23:24,497 nipype.workflow DEBUG:
	 [MultiProc] Starting (n_procs=30, mem_gb=907.02, cwd=/LOCAL/jzerbe/code/ICA-fMRI/notebooks)
220511-18:23:24,695 nipype.workflow DEBUG:
	 Creating flat graph for workflow: dataset_wf
220511-18:23:24,746 nipype.workflow DEBUG:
	 expanding workflow: dataset_wf
220511-18:23:24,747 nipype.workflow DEBUG:
	 processing node: dataset_wf.node_1
220511-18:23:24,752 nipype.workflow DEBUG:
	 expanding workflow: dataset_wf.node_1
220511-18:23:24,752 nipype.workflow DEBUG:
	 processing node: melodicwf.inputspec
220511-18:23:24,753 nipype.workflow DEBUG:
	 processing node: melodicwf.calcthresh
220511-18:23:24,753 nipype.workflow DEBUG:
	 processing node: melodicwf.susan
220511-18:23:24,754 nipype.workflow DEBUG:
	 processing node: melodicwf.tfilt
220511-18:23:24,755 nipype.workflow DEBUG:
	 processing node: melodicwf.melodic
220511-18:23:24,755 nipype.workflow DEBUG:
	 finished expanding workflow: dataset_wf.node_1
220511-18:23:24,756 nipype.workflow DEBUG

220511-18:23:24,807 nipype.workflow DEBUG:
	 expanding workflow: dataset_wf.node_13
220511-18:23:24,808 nipype.workflow DEBUG:
	 processing node: melodicwf.inputspec
220511-18:23:24,808 nipype.workflow DEBUG:
	 processing node: melodicwf.calcthresh
220511-18:23:24,809 nipype.workflow DEBUG:
	 processing node: melodicwf.susan
220511-18:23:24,809 nipype.workflow DEBUG:
	 processing node: melodicwf.tfilt
220511-18:23:24,810 nipype.workflow DEBUG:
	 processing node: melodicwf.melodic
220511-18:23:24,810 nipype.workflow DEBUG:
	 finished expanding workflow: dataset_wf.node_13
220511-18:23:24,811 nipype.workflow DEBUG:
	 processing node: dataset_wf.node_14
220511-18:23:24,811 nipype.workflow DEBUG:
	 expanding workflow: dataset_wf.node_14
220511-18:23:24,812 nipype.workflow DEBUG:
	 processing node: melodicwf.inputspec
220511-18:23:24,813 nipype.workflow DEBUG:
	 processing node: melodicwf.calcthresh
220511-18:23:24,813 nipype.workflow DEBUG:
	 processing node: melodicwf.susan
220511-18:23:2

220511-18:23:24,859 nipype.workflow DEBUG:
	 processing node: melodicwf.tfilt
220511-18:23:24,860 nipype.workflow DEBUG:
	 processing node: melodicwf.melodic
220511-18:23:24,860 nipype.workflow DEBUG:
	 finished expanding workflow: dataset_wf.node_25
220511-18:23:24,861 nipype.workflow DEBUG:
	 processing node: dataset_wf.node_26
220511-18:23:24,861 nipype.workflow DEBUG:
	 expanding workflow: dataset_wf.node_26
220511-18:23:24,862 nipype.workflow DEBUG:
	 processing node: melodicwf.inputspec
220511-18:23:24,862 nipype.workflow DEBUG:
	 processing node: melodicwf.calcthresh
220511-18:23:24,863 nipype.workflow DEBUG:
	 processing node: melodicwf.susan
220511-18:23:24,863 nipype.workflow DEBUG:
	 processing node: melodicwf.tfilt
220511-18:23:24,863 nipype.workflow DEBUG:
	 processing node: melodicwf.melodic
220511-18:23:24,864 nipype.workflow DEBUG:
	 finished expanding workflow: dataset_wf.node_26
220511-18:23:24,864 nipype.workflow DEBUG:
	 processing node: dataset_wf.node_27
220511-18

220511-18:23:24,984 nipype.workflow DEBUG:
	 [Node] calcthresh - setting input maskfile = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-effort_space-T1w_desc-brain_mask.nii.gz
220511-18:23:24,984 nipype.workflow DEBUG:
	 [Node] melodic - setting input mask = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-effort_space-T1w_desc-brain_mask.nii.gz
220511-18:23:24,985 nipype.workflow DEBUG:
	 [Node] susan - setting input fwhm = 4.0
220511-18:23:24,985 nipype.workflow DEBUG:
	 [Node] tfilt - setting input highpass_sigma = 80.0
220511-18:23:24,986 nipype.workflow DEBUG:
	 [Node] melodic - setting input out_dir = /LOCAL/jzerbe/temp_results/melodic/sub-28/sub-28_ses-None_task-effort_run-None_space-T1w-melodic
220511-18:23:24,986 nipype.workflow DEBUG:
	 [Node] melodic - setting input tr_sec = 1.5
220511-18:23:24,987 nipype.workflow DEBUG:
	 Removed the identity node dataset_wf.melodicwf.inputspec from the graph.
220511-1

220511-18:23:25,11 nipype.workflow DEBUG:
	 [Node] calcthresh - setting input boldfile = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-23/func/sub-23_task-effort_space-T1w_desc-preproc_bold.nii.gz
220511-18:23:25,12 nipype.workflow DEBUG:
	 [Node] susan - setting input in_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-23/func/sub-23_task-effort_space-T1w_desc-preproc_bold.nii.gz
220511-18:23:25,12 nipype.workflow DEBUG:
	 [Node] calcthresh - setting input maskfile = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-23/func/sub-23_task-effort_space-T1w_desc-brain_mask.nii.gz
220511-18:23:25,13 nipype.workflow DEBUG:
	 [Node] melodic - setting input mask = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-23/func/sub-23_task-effort_space-T1w_desc-brain_mask.nii.gz
220511-18:23:25,13 nipype.workflow DEBUG:
	 [Node] susan - setting input fwhm = 4.0
220511-18:23:25,13 nipype.workflow DEBUG:
	 [Node] tfilt - setting input 

...

RuntimeError: Traceback (most recent call last):
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 148, in __init__
    mp_context=mp_context,
TypeError: __init__() got an unexpected keyword argument 'initializer'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node
    result["result"] = node.run(updatehash=updatehash)
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run
    result = self._run_interface(execute=True)
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface
    return self._run_command(execute)
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 751, in _run_command
    f"Exception raised w

220511-18:24:01,625 nipype.workflow INFO:
	 [Node] Finished "susan", elapsed time 32.065375s.
220511-18:24:01,638 nipype.workflow DEBUG:
	 Needed files: /tmp/tmpf7iz5u7r/dataset_wf/melodicwf/susan/sub-18_task-effort_space-T1w_desc-preproc_bold_smooth.nii.gz;/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-18/func/sub-18_task-effort_space-T1w_desc-preproc_bold.nii.gz;/tmp/tmpf7iz5u7r/dataset_wf/melodicwf/susan/_0x01ebd82e0fd5f9451ec47d68594c504c_unfinished.json;/tmp/tmpf7iz5u7r/dataset_wf/melodicwf/susan/command.txt;/tmp/tmpf7iz5u7r/dataset_wf/melodicwf/susan/_inputs.pklz;/tmp/tmpf7iz5u7r/dataset_wf/melodicwf/susan/_node.pklz
220511-18:24:01,640 nipype.workflow DEBUG:
	 Needed dirs: /tmp/tmpf7iz5u7r/dataset_wf/melodicwf/susan/_report
220511-18:24:01,642 nipype.workflow DEBUG:
	 Removing files: 
220511-18:24:01,645 nipype.workflow DEBUG:
	 Saving results file: '/tmp/tmpf7iz5u7r/dataset_wf/melodicwf/susan/result_susan.pklz'
220511-18:24:01,664 nipype.workflow DEBUG:
	 [Node

	 [Node] Writing post-exec report to "/tmp/tmpbtrrlcmw/dataset_wf/melodicwf/tfilt/_report/report.rst"
220511-18:24:13,573 nipype.workflow INFO:
	 [Node] Finished "susan", elapsed time 42.168875s.
220511-18:24:13,580 nipype.workflow DEBUG:
	 Needed files: /tmp/tmpf5qbnh2_/dataset_wf/melodicwf/susan/sub-08_task-effort_space-MNI152NLin6Asym_desc-preproc_bold_smooth.nii.gz;/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-08/func/sub-08_task-effort_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz;/tmp/tmpf5qbnh2_/dataset_wf/melodicwf/susan/_0xe6548835f422250c117b6cbfbbff3c9d_unfinished.json;/tmp/tmpf5qbnh2_/dataset_wf/melodicwf/susan/command.txt;/tmp/tmpf5qbnh2_/dataset_wf/melodicwf/susan/_inputs.pklz;/tmp/tmpf5qbnh2_/dataset_wf/melodicwf/susan/_node.pklz
220511-18:24:13,582 nipype.workflow DEBUG:
	 Needed dirs: /tmp/tmpf5qbnh2_/dataset_wf/melodicwf/susan/_report
220511-18:24:13,597 nipype.workflow DEBUG:
	 Removing files: 
220511-18:24:13,599 nipype.workflow DEBUG:
	 Saving re

	 Needed files: /tmp/tmp5dah9jjk/dataset_wf/melodicwf/tfilt/sub-28_task-1back_space-MNI152NLin2009cAsym_desc-preproc_bold_smooth_filt.nii.gz;/tmp/tmpkc5i8q4w/dataset_wf/melodicwf/susan/sub-28_task-1back_space-MNI152NLin2009cAsym_desc-preproc_bold_smooth.nii.gz;/tmp/tmp5dah9jjk/dataset_wf/melodicwf/tfilt/_0x60e740886f68af72b81d077dd271a59a_unfinished.json;/tmp/tmp5dah9jjk/dataset_wf/melodicwf/tfilt/command.txt;/tmp/tmp5dah9jjk/dataset_wf/melodicwf/tfilt/_inputs.pklz;/tmp/tmp5dah9jjk/dataset_wf/melodicwf/tfilt/_node.pklz
220511-18:24:20,104 nipype.workflow DEBUG:
	 Needed dirs: /tmp/tmp5dah9jjk/dataset_wf/melodicwf/tfilt/_report
220511-18:24:20,106 nipype.workflow DEBUG:
	 Removing files: 
220511-18:24:20,108 nipype.workflow DEBUG:
	 Saving results file: '/tmp/tmp5dah9jjk/dataset_wf/melodicwf/tfilt/result_tfilt.pklz'
220511-18:24:20,110 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/tmp/tmp5dah9jjk/dataset_wf/melodicwf/tfilt/_report/report.rst"
220511-18:24:20,358 nipype.w

	 Needed dirs: /tmp/tmpyojiqy8v/dataset_wf/melodicwf/susan/_report220511-18:24:24,300 nipype.workflow DEBUG:
	 Saving results file: '/tmp/tmpx24hsvno/dataset_wf/melodicwf/susan/result_susan.pklz'

220511-18:24:24,301 nipype.workflow DEBUG:
	 Removing files: 
220511-18:24:24,302 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/tmp/tmpx24hsvno/dataset_wf/melodicwf/susan/_report/report.rst"220511-18:24:24,303 nipype.workflow DEBUG:
	 Saving results file: '/tmp/tmpyojiqy8v/dataset_wf/melodicwf/susan/result_susan.pklz'

220511-18:24:24,307 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/tmp/tmpyojiqy8v/dataset_wf/melodicwf/susan/_report/report.rst"
220511-18:24:46,969 nipype.workflow INFO:
	 [Node] Finished "tfilt", elapsed time 53.585746s.
220511-18:24:46,979 nipype.workflow DEBUG:
	 Needed files: /tmp/tmp0zkwwzrz/dataset_wf/melodicwf/tfilt/sub-28_task-effort_space-T1w_desc-preproc_bold_smooth_filt.nii.gz;/tmp/tmp5ytpus2z/dataset_wf/melodicwf/susan/sub-28_task-ef

In [72]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/LOCAL/jzerbe/code/ICA-fMRI/notebooks/crash-20220511-182359-jzerbe-melodic-98583b0e-1b16-44be-9d57-3452e823967c.pklz') 

220511-18:51:17,466 nipype.utils DEBUG:
	 Loading pkl: /LOCAL/jzerbe/code/ICA-fMRI/notebooks/crash-20220511-182359-jzerbe-melodic-98583b0e-1b16-44be-9d57-3452e823967c.pklz


In [75]:
res

{'node': dataset_wf.melodicwf.melodic,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 148, in __init__\n    mp_context=mp_context,\n',
  "TypeError: __init__() got an unexpected keyword argument 'initializer'\n",
  '\nDuring handling of the above exception, another exception occurred:\n\n',
  'Traceback (most recent call last):\n',
  '  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/LOCAL/jzerbe/testenv/lib/pytho